In [1]:
from bs4 import BeautifulSoup
from lxml import html
import requests
from urllib2 import urlopen
import urllib2,cookielib
import pandas as pd
import nba_py as nba
from nba_py import player
import sys

In [12]:
from bs4 import BeautifulSoup
from lxml import html
import requests
from urllib2 import urlopen
import urllib2,cookielib
import pandas as pd
import nba_py as nba
from nba_py import player
import sys


def makeSoup(url):
	hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
	       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
	       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
	       'Accept-Encoding': 'none',
	       'Accept-Language': 'en-US,en;q=0.8',
	       'Connection': 'keep-alive'}
	req = urllib2.Request(url, headers=hdr)
	try:
	    page = urllib2.urlopen(req)
	except urllib2.HTTPError, e:
	    print e.fp.read()
	content = page.read()

	soup = BeautifulSoup(content, 'lxml')
	return soup


def makeHeaders(soup):

	column_headers = [th.getText() for th in soup.findAll('th')]
	return  column_headers

def makeDataframe(soup):

	column_headers = makeHeaders(soup)
	data_rows = soup.findAll('tr')[1:]
	player_data = [[td.getText() for td in data_rows[i].findAll('td')] for i in range(len(data_rows))]

	df = pd.DataFrame(player_data, columns=column_headers)
	return df


def getPlayerId(name):
    name = name.split(' ')

    # If returned with execption 400 bad request for url, you sometimes have to open the link in your browser then it will work. Weird bug.

    player = nba.player.get_player(name[0], name[1], just_id=True)
    return player

def getURL(id, season):

	id = str(int(id))
	url ='http://nbasavant.com/ajax/getShotsByPlayer.php?hfST=&hfQ=&pid%5B%5D='+id+'&hfSZB=&hfSZA=&hfSZR=&ddlYear='+season+'&txtGameDateGT=&txtGameDateLT=&ddlGameTimeGT_min=&ddlGameTimeGT_sec=&ddlGameTimeLT_min=&ddlGameTimeLT_sec=&ddlShotClockGT=&ddlShotClockLT=&ddlDefDistanceGT=&ddlDefDistanceLT=&ddlDribblesGT=&ddlDribblesLT=&ddlTouchTimeGT=&ddlTouchTimeLT=&ddlShotDistanceGT=&ddlShotDistanceLT=&ddlTeamShooting=&ddlTeamDefense=&hfPT=&ddlGroupBy=player&ddlOrderBy=shots_made_desc&hfGT=0%7C&ddlShotMade=&ddlMin=0&player_id='+id+'&data=null&_=1459806001585'
    
	return url

def getData(name, season):
	id = getPlayerId(name)
	url = getURL(id, season)
	soup = makeSoup(url)
	df = makeDataframe(soup)
	return df


if __name__ == '__main__':
    df = getData('stephen curry', '2014')
    print df.head()



    Game Date         Player               Opp.  Q   Time          Shot Type  \
0  2014-12-13  Stephen Curry   Dallas Mavericks  3   2:25  Driving Bank shot   
1  2015-02-20  Stephen Curry  San Antonio Spurs  3  10:41  Driving Bank shot   
2  2014-12-13  Stephen Curry   Dallas Mavericks  4   7:05  Driving Bank shot   
3  2015-03-21  Stephen Curry          Utah Jazz  2   0:34  Driving Bank shot   
4  2015-04-04  Stephen Curry   Dallas Mavericks  1  11:23  Driving Bank shot   

  Shot Dist. Made? Drib. Shot Clock Touch Time         Defender Def Dist.  \
0    5.20ft.   Yes     8      16.00       6.30   Nowitzki, Dirk      3.20   
1   13.60ft.   Yes     5       2.90       4.50     Parker, Tony      1.70   
2    6.00ft.   Yes     0       9.30       0.80  Chandler, Tyson      3.70   
3    9.00ft.   Yes    10      15.30       8.70  Favors, Derrick      2.30   
4    4.10ft.   Yes     0      20.80       0.80   Nowitzki, Dirk      2.50   

               
0  \n\n\n  \n  
1  \n\n\n  \n  
2  \n\n\